In [105]:
# import load_model
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd

In [106]:
# load AI model from Models
model = load_model("./Results/progress/Fourth_model/models/model_20230112-180708_final.h5")

In [107]:
# read data from file into pandas dataframe
df = pd.read_csv("./Data/dataset/VETUSDT.csv")


LOOKBACK_WINDOW = 15

# create sequences from dataframe
sequences = []
for i in range(LOOKBACK_WINDOW - 1, len(df)):
    sequences.append(df[i:i+LOOKBACK_WINDOW])

# create numpy array from sequences
sequences = np.array(sequences)




/tmp/ipykernel_2761910/4117980202.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences = np.array(sequences)


In [108]:
first_item = sequences[0]
first_item = pd.DataFrame(first_item)

original_items = ['event_time','original_open', 'original_close', 'original_high', 'original_low', 'original_volume']


# create nice comment that explains what this function does

# Receives an index and returns the data for the training and for the environment
def get_train_env_data(data, index):
    item = pd.DataFrame(data[i])
    return item[original_items], item.drop(original_items, axis=1)

In [109]:
from matplotlib import pyplot as plt
# plot data from csv file
def plot_data(data):
    plt.plot(data)
    plt.xlabel("Episode")
    plt.legend(["Average Reward"])
    plt.show()

training_history = pd.read_csv("./Results/progress/Fourth_model/reward_history_20230112-180708.csv")
print(training_history)

plot_data(training_history['average_reward'])

    average_reward   epsilon  balance   episode  time_train  steps
0      -629.809005  0.931000        0  3.447417         129    NaN
1      -491.603222  0.884450        1  0.936605         133    NaN
2      -318.728680  0.840227        2  0.964283         117    NaN
3      -206.134153  0.798216        3  0.984694         136    NaN
4      -138.114934  0.758305        4  0.946212         152    NaN
5      -112.085894  0.720390        5  0.901414         180    NaN
6       -85.287294  0.684371        6  0.946668         156    NaN
7       -54.420339  0.650152        7  0.973154         194    NaN
8       -39.759302  0.617644        8  0.978177         209    NaN
9       -20.314286  0.586762        9  1.014463         192    NaN
10       -5.752975  0.557424       10  0.874133         219    NaN
11        3.512877  0.529553       11  0.987288         247    NaN
12       19.710523  0.503075       12  0.886416         257    NaN
13       43.751553  0.477921       13  0.958568         230   

In [94]:
from enum import Enum
from tqdm import tqdm
class ActionSpace(Enum):
    BUY = 0
    HOLD = 1
    SELL = 2


# loop through all sequences and predict buy hold or sell and put the results on a graph
buy_sells = []
for i in tqdm(range(0, len(sequences[:300]))):
    train, env = get_train_env_data(sequences, i)
    env = env.to_numpy()
    prediction = model.predict(env.reshape(1, 15, 17), verbose=0)
    action = ActionSpace(np.argmax(prediction[0]))
    buy_sells.append((train.iloc[-1], action, i))

# create a graph with the buy and sell points candlesticks




100%|██████████| 300/300 [00:11<00:00, 26.14it/s]


In [ ]:
print(buy_sells)

In [97]:
def plot_buy_sell(data):
    plt.figure()

    width = .4
    width2 = .04

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i in range(0, len(data)):
        item = data[i]
        item_candlestick = item[0]
        action = item[1]
        index = item[2]

        if action == ActionSpace.BUY:
            plt.plot(index, item_candlestick['original_close'], 'y+')
        elif action == ActionSpace.SELL:
            plt.plot(index, item_candlestick['original_close'], 'b-')

        if item_candlestick['original_close'] >= item_candlestick['original_open']:
            height = item_candlestick['original_close'] - item_candlestick['original_open']
            height_max = item_candlestick['original_high'] - item_candlestick['original_low']
            plt.bar(index, height, width, item_candlestick['original_open'], color=col1)
            plt.bar(index, height_max, width2, item_candlestick['original_open'], color=col1)
        else:
            height = item_candlestick['original_open'] - item_candlestick['original_close']
            height_max = item_candlestick['original_high'] - item_candlestick['original_low']
            plt.bar(index, height, width,  item_candlestick['original_close'], color=col2)
            plt.bar(index, height_max, width2,  item_candlestick['original_close'], color=col2)

    plt.ylabel("Price")

    plt.xlabel("Minutes")

    plt.show()


%matplotlib tk
plot_buy_sell(buy_sells)




In [53]:
%load_ext tensorboard
%tensorboard --logdir Training/logs/fit